# Classificação Textual Multirótulo

In [13]:
import pandas as pd
import numpy as np
import nltk
from sklearn.linear_model import LogisticRegression

In [14]:
# Algumas funções utilizadas indiretamente apresentam avisos de "deprecation", portanto silenciamos esses avisos.

import warnings
warnings.filterwarnings("ignore")

In [15]:
train = pd.read_csv('q2_data/train.csv')
test = pd.read_csv('q2_data/test.csv')

Vejamos um comentário interessante como exemplo.

In [16]:
train.iloc[6]

id                                           0002bcb3da6cb337
comment_text     COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
toxic                                                       1
severe_toxic                                                1
obscene                                                     1
threat                                                      0
insult                                                      1
identity_hate                                               0
Name: 6, dtype: object

Esse comentário foi, simultâneamente, classificado como:

- `toxic`: comentário ofensivo ou desrespeitoso
- `severe_toxic`: comentário extremamente ofensivo ou desrespeitoso
- `obscene`: comentário sexual, vulgar ou obsceno
- `insult`: comentário insultante ou desrespeitoso

Vejamos as primeiras 5 observações do conjunto de teste.

In [17]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


Separamos os dados de treino e teste em `X` e `y`, onde `X` são os comentários e `y` são as classificações.

In [18]:
X_train = train['comment_text']
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

Carregamos os pacotes de stopwords, de stemmer e de tokenização.

In [19]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Downloading the necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Definimos uma função de preprocessamento. Por conveniência, documentamos, em inglês, seu funcionamento em usa `docstring`.

In [20]:
def preprocess_text(text):
    '''
    Takes in a string of text, then performs the following:
    1. Tokenizes the sentence into words
    2. Removes non-alphabetic characters
    3. Converts all characters to lowercase
    4. Removes stopwords
    5. Stems the words
    6. Joins the stemmed words back into one string
    '''
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    stemmer = SnowballStemmer("english")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return " ".join(stemmed_tokens)

Para cada comentário, aplicamos a função de preprocessamento.

In [21]:
# Preprocessing the training data
X_train_preprocessed = [preprocess_text(text) for text in X_train]

Instanciamos então o vetorizador `TfidfVectorizer` e o ajustamos aos dados de treino.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fitting the vectorizer to the training data
X_train_transformed = vectorizer.fit_transform(X_train_preprocessed)

Separamos os dados de treino em treino e validação usando `train_test_split`.

In [23]:
# Split the training data into a training and validation set
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_transformed,
                                                                          y_train,
                                                                          test_size=0.2,
                                                                          random_state=42)

Como primeiro modelo, utilizaremos o `MultiOutputClassifier` com o `LogisticRegression`.

In [24]:
# Importing the multi-label classifier
from sklearn.multioutput import MultiOutputClassifier

# Creating the multi-label classifier
multi_clf = MultiOutputClassifier(LogisticRegression())

# Training the multi-label classifier
multi_clf.fit(X_train_split, y_train_split)

# Predicting on the validation set
y_val_pred = multi_clf.predict(X_val_split)

Vejamos as métricas de classificação para cada uma das classes usando o `classification_report`.

In [25]:
# Classification report
from sklearn.metrics import classification_report

print(classification_report(y_val_split, y_val_pred))

              precision    recall  f1-score   support

           0       0.91      0.60      0.73      3056
           1       0.58      0.23      0.33       321
           2       0.92      0.61      0.73      1715
           3       0.75      0.12      0.21        74
           4       0.82      0.48      0.60      1614
           5       0.77      0.15      0.25       294

   micro avg       0.88      0.54      0.67      7074
   macro avg       0.79      0.37      0.48      7074
weighted avg       0.87      0.54      0.66      7074
 samples avg       0.05      0.05      0.05      7074



Vamos analisar uma previsão em específico, a do comentário que vimos no início.

Relembramos o comentário original.

In [27]:
X_train.iloc[6]

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

Após o preprocessamento, o comentário fica assim.

In [31]:
X_train_preprocessed[6]

'cocksuck piss around work'

A previsão do modelo agora desconsidera a segunda categoria, que é a de `severe_toxic`.

In [57]:
# Predicting on the validation set
y_val_pred = multi_clf.predict(X_train_transformed)

print(y_val_pred[6])

[1 0 1 0 1 0]
